In [247]:
import os.path
import pandapower.networks as pn
import pandapower as pp
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from numpy import flatnonzero as find, ones, zeros, Inf, pi, exp, conj, r_, arange, array
from pypower_.makeYbus import makeYbus
from pypower_.idx_brch import F_BUS, T_BUS, RATE_A, PF, QF, PT, QT, MU_SF, MU_ST
from pypower_.idx_gen import GEN_BUS, PG, QG
from pypower_.idx_cost import MODEL, PW_LINEAR, NCOST, POLYNOMIAL
from pypower_.totcost import totcost
from pypower_.makeSbus import makeSbus
from pypower_.idx_bus import BUS_TYPE, REF, VM, VA, MU_VMAX, MU_VMIN, LAM_P, LAM_Q
import cvxpy as cp
import numpy as np
from scipy.sparse import csr_matrix as sparse
from pypower_.idx_bus import PD, QD
from pypower_.idx_gen import GEN_BUS, PG, QG, GEN_STATUS

In [248]:
import sys
print(sys.path)
print(os.path)
# sys.path.insert(1, '/home/ubuntu-h/PycharmProjects/scientificProject')

['/snap/pycharm-professional/314/plugins/python/helpers-pro/jupyter_debug', '/home/ubuntu-h/PycharmProjects/scientificProject', '/home/ubuntu-h/PycharmProjects/scientificProject', '/snap/pycharm-professional/314/plugins/python/helpers/pydev', '/home/ubuntu-h/PycharmProjects/scientificProject/notebooks', '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python310.zip', '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10', '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/lib-dynload', '', '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/site-packages', '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg']
<module 'posixpath' from '/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/posixpath.py'>


## 使用pandapower 构建不同电网结构的OPF模型

In [249]:
import pandapower as pp
net = pp.create_empty_network()

# create buses
bus1 = pp.create_bus(net, vn_kv=110.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)
# bus5 = pp.create_bus(net, vn_kv=110.)

# create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type="149-AL1/24-ST1A 110.0")
pp.create_line(net, bus4, bus2, length_km=40., std_type="149-AL1/24-ST1A 110.0")
pp.create_line(net, bus1, bus2, length_km=70., std_type='149-AL1/24-ST1A 110.0')

# create loads
pp.create_load(net, bus2, p_mw=60., controllable=False)
pp.create_load(net, bus3, p_mw=70., controllable=False)
pp.create_load(net, bus4, p_mw=25., controllable=False)

# create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=0, max_p_mw=1000, vm_pu=1.05)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80, vm_pu=1.00, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.00, controllable=True)


costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=20)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

net.bus["min_vm_pu"] = 0.98
net.bus["max_vm_pu"] = 1.02
net.line["max_loading_percent"] = 50

![xx -w50](grid2GUROBI.png)

### 也可使用典型的IEEE 电网模型

In [250]:
# # net.sn_mva = 100
net = pp.networks.case14()
net = pp.networks.case_ieee30()

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,20.0,0.0,0.0,0.0,0.0
1,0,gen,0.0,10.0,0.0,0.0,0.0,0.0
2,1,gen,0.0,10.0,0.0,0.0,0.0,0.0


In [251]:
om,ppopt = pp.runopp(net, delta=1e-16)   # solve the OPF problem

In [254]:
# the total cost of generation
net.res_cost

1570.2048160492498

In [256]:
## unpack data
ppc = om.get_ppc()
baseMVA, bus, gen, branch, gencost = \
    ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"], ppc["gencost"]
vv, _, _, _ = om.get_idx()

## problem dimensions
nb = bus.shape[0]          ## number of buses
nl = branch.shape[0]       ## number of branches
ny = om.getN('var', 'y')   ## number of piece-wise linear costs, 'y' is a part of p-w linear costs

## bounds on optimization vars
x0, xmin, xmax = om.getv()

In [257]:
## build admittance matrices
Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

/home/ubuntu-h/PycharmProjects/scientificProject/pypower_/makeYbus.py:49: ComplexWarning: Casting complex values to real discards the imaginary part
  tap[i] = branch[i, TAP]  ## assign non-zero tap ratios
/home/ubuntu-h/anaconda3/envs/scientificProject/lib/python3.10/site-packages/scipy/sparse/_coo.py:160: ComplexWarning: Casting complex values to real discards the imaginary part
  self.col = np.array(col, copy=copy, dtype=idx_dtype)


In [258]:
## Set the lower and upper bound for all variables
ll, uu = xmin.copy(), xmax.copy()
# print(f'll : {ll}; uu : {uu}')
ll[xmin == -Inf] = -1e10   ## replace Inf with numerical proxies
uu[xmax ==  Inf] =  1e10
Va_refs = bus[bus[:, BUS_TYPE]  == REF, VA] * (pi / 180)
# print(f"Va_refs: {Va_refs}")
ll[vv["i1"]["Va"]:vv["iN"]["Va"]] = -np.ones_like(bus[:, VA]) * (pi / 2) # Va lower bound 赋值
uu[vv["i1"]["Va"]:vv["iN"]["Va"]] = np.ones_like(bus[:, VA]) * (pi / 2) # Va upper bound 赋值
## deal with the Va_refs
ll[vv["i1"]["Va"]:vv["iN"]["Va"]][bus[:, BUS_TYPE]  == REF] = Va_refs  # Va_refs lower bound 赋值
uu[vv["i1"]["Va"]:vv["iN"]["Va"]][bus[:, BUS_TYPE]  == REF] = Va_refs # Va_refs upper bound 赋值
## deal with reactive power, whose ll and uu is 1e9
ll[ll<-1e4] = -100
uu[uu>1e4] = 100
print(f'll"{ll};\n uu:{uu}')

ll"[ 0.00000000e+00 -1.57079633e+00 -1.57079633e+00 -1.57079633e+00
  1.05000000e+00  9.80000000e-01  9.80000000e-01  9.80000000e-01
 -1.00000000e-16 -1.00000000e-16 -1.00000000e-16 -1.00000000e+02
 -1.00000000e+02 -1.00000000e+02];
 uu:[   0.            1.57079633    1.57079633    1.57079633    1.05
    1.02          1.02          1.02       1000.           80.
  100.          100.          100.          100.        ]


In [261]:
x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]] = bus[:, VM] # 赋值

In [262]:
Va_refs = bus[bus[:, BUS_TYPE]  == REF, VA] * (pi / 180)
x0[vv["i1"]["Va"]:vv["iN"]["Va"]] = bus[:, VA] * (pi / 180) # 赋值

In [263]:
## 赋值
x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]] = gen[:,PG]
x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]] = gen[:,QG]


### 处理cost function

In [264]:
## grab Pg & Qg
Pg = x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]]  ## active generation in p.u.
Qg = x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]]  ## reactive generation in p.u.

##----- evaluate objective function -----
## polynomial cost of P and Q
# use totcost only on polynomial cost in the minimization problem formulation, pwl cost is the sum of the y variables.
ipol = find(gencost[:, MODEL] == POLYNOMIAL)   ## poly MW and MVAr costs
xx = r_[ Pg, Qg ] * baseMVA
if len(ipol)>0:
    f = sum( totcost(gencost[ipol, :], xx[ipol]) )  ## cost of poly P or Q
else:
    f = 0

In [267]:
First_Or_Con = 4
print(f"{gencost[ipol, First_Or_Con], gencost[:, First_Or_Con]}")

(array([20., 10., 10.]), array([20., 10., 10.]))


### 处理约束部分

In [269]:
## grab Pg & Qg
Pg = x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]]  ## active generation in p.u.
Qg = x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]]  ## reactive generation in p.u.

In [270]:
## put Pg & Qg back in gen
gen[:, PG] = Pg * baseMVA  ## active generation in MW
gen[:, QG] = Qg * baseMVA  ## reactive generation in MVAr

In [271]:
## reconstruct V
Va = x0[vv["i1"]["Va"]:vv["iN"]["Va"]]
Vm = x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]]
V = Vm * exp(1j * Va)

on = find(gen[:, GEN_STATUS] > 0)      ## which generators are on?
gbus = gen[on, GEN_BUS]                   ## what buses are they at?

## form net complex bus power injection vector
nb = bus.shape[0]
ngon = on.shape[0]
## connection matrix, element i, j is 1 if gen on(j) at bus i is ON
Cg = sparse((ones(ngon), (gbus, range(ngon))), (nb, ngon))

In [ ]:
## 不等式约束： 线电压的功率
ppopt = {}
il = find(branch[:, RATE_A] < 1e10)
nl2 = len(il)           ## number of constrained lines
if any(net.line['max_i_ka']) > 0:
    ppopt['OPF_FLOW_LIM'] = 2

if nl2 > 0:
    flow_max = (branch[il, RATE_A] / baseMVA)**2
    flow_max[flow_max == 0] = 1e5
    print(f"flow_max: {flow_max}")
    if ppopt['OPF_FLOW_LIM'] == 2:      ## current magnitude limit, |I|
        If = Yf * V
        It = Yt * V
        h = r_[ If * conj(If) - flow_max,     ## branch I limits (from bus)
                It * conj(It) - flow_max ].real    ## branch I limits (to bus)
    else:
        ## compute branch power flows
        ## complex power injected at "from" bus (p.u.)
        Sf = V[ branch[il, F_BUS].astype(int) ] * conj(Yf * V)
        ## complex power injected at "to" bus (p.u.)
        St = V[ branch[il, T_BUS].astype(int) ] * conj(Yt * V)
        if ppopt['OPF_FLOW_LIM'] == 1:   ## active power limit, P (Pan Wei)
            h = r_[ Sf.real**2 - flow_max,   ## branch P limits (from bus)
                    St.real**2 - flow_max ]  ## branch P limits (to bus)
        else:                ## apparent power limit, |S|
            h = r_[ Sf * conj(Sf) - flow_max, ## branch S limits (from bus)
                    St * conj(St) - flow_max ].real  ## branch S limits (to bus)
else:
    h = zeros((0,1))

## 使用GUROBI 构建优化模型

In [291]:
# Parameters
# obtain the complex load
Pd = bus[:, PD]
Qd = bus[:, QD]

lb_update = -1e4

# obtain the range of voltage; the range is from the setting
V_range = 0.02
V_ll,V_uu = 1.0-V_range, 1.0+V_range

# obtain the power range
Pg_lower, Pg_upper = ll[vv["i1"]["Pg"]:vv["iN"]["Pg"]], uu[vv["i1"]["Pg"]:vv["iN"]["Pg"]]*baseMVA
Qg_lower, Qg_upper = ll[vv["i1"]["Qg"]:vv["iN"]["Qg"]], uu[vv["i1"]["Qg"]:vv["iN"]["Qg"]]*baseMVA

# the dimension of the OPF
nb = bus.shape[0]          ## number of buses
nl = branch.shape[0]       ## number of branches

# create the model; set the model type: NonConvex.
model = gp.Model("PowerGeneration");model.setParam('NonConvex', 2)

# creat variables for the model; the numbers in the following codes should be replaced by parameters
V_re = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='V real')
V_im = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='V imag')
Pg = model.addMVar(ngon, lb=Pg_lower, ub=Pg_upper, vtype=GRB.CONTINUOUS, name='Pg')
Qg = model.addMVar(ngon, lb=Qg_lower, ub=Qg_upper, vtype=GRB.CONTINUOUS, name='Qg')
temp1 = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 1')
temp2 = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 2')
Pg_net = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Pg net')  # the number of buses, too
Qg_net = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Qg net')
Line_temp_1 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 1')    # the shape should be line number
Line_temp_2 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 2')    # the shape should be line number
Line_temp_3 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 3')    # the shape should be line number
Line_temp_4 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 4')    # the shape should be line number

# the prameters of branches
G = np.real(Ybus)
B = np.imag(Ybus)
# the parameters for currents in branches
G_yf, B_yf = np.real(Yf), np.imag(Yf)
G_yt, B_yt = np.real(Yt), np.imag(Yt)

# Constraints; insert the formulas
Temp_c1 = model.addConstr(G@V_re - B@V_im == temp1, 'Temp Constr 1')
Temp_c2 = model.addConstr(G @ V_im + B @ V_re == temp2, 'Temp Constr 2')
Pg_net_c = model.addConstr(Cg @ Pg - Pd == Pg_net, 'Pg net Constr')
Qg_net_c = model.addConstr(Cg @ Qg - Qd == Qg_net, 'Qg net Constr')
# print(f'Pd:{Pd}\n Qd:{Qd}')
CVL_CPL = model.addConstrs((V_re[i] * temp1[i] *baseMVA+ V_im[i] * temp2[i]*baseMVA == Pg_net[i]  for i in range(nb)), 'Cvl P')  # break into different parts
CVL_CPL_Q = model.addConstrs((V_im[i] * temp1[i] *baseMVA - V_re[i] * temp2[i]*baseMVA == Qg_net[i] for i in range(nb)), 'Cvl Q')

# the constraints of voltages on all buses; set the constraint for slack bus, then give the upper and lower constraints for other buses
V_slack = model.addConstr(V_re[0]==1.05, 'V slack real')
V_slack_a = model.addConstr(V_im[0]==0, 'V slack imag')
# the upper&lower constraints for buses except for the slack bus
V_upper = model.addConstrs((V_re[i]**2 + V_im[i]**2 <= V_uu**2 for i in range(1,nb)), 'V upper bound')
V_lower = model.addConstrs((V_re[i]**2 + V_im[i]**2 >= V_ll**2 for i in range(1,nb)), 'V lower bound')

# Yf * W * Yf; the number is equal to that of the lines/branches
Line_temp_c_1 = model.addConstr(G_yf@V_re - B_yf@V_im == Line_temp_1, 'Branch Con 1')
Line_temp_c_2 = model.addConstr(G_yf @ V_im + B_yf @ V_re == Line_temp_2, 'Branch Con 2')
Line_temp_c_3 = model.addConstr(G_yt@V_re - B_yt@V_im == Line_temp_3, 'Branch Con 3')
Line_temp_c_4 = model.addConstr(G_yt @ V_im + B_yt @ V_re == Line_temp_4, 'Branch Con 4')
Line_c_1 = model.addConstrs((Line_temp_1[i]**2 + Line_temp_2[i]**2 <= np.real(flow_max[i]) for i in range(nl)), 'Branch Con 1') # p.u.
Line_c_2 = model.addConstrs((Line_temp_3[i]**2 + Line_temp_4[i]**2 <= np.real(flow_max[i]) for i in range(nl)), 'Branch Con 2') # p.u.

# set the objective of the model
print('ipol:', ipol)
# The "gencost" changes with different cases. Keep in Mind!
# model.setObjective(gencost[ipol, First_Or_Con]@Pg**2+gencost[ipol, First_Or_Con+1]@Pg, GRB.MINIMIZE)
model.setObjective(gencost[ipol, First_Or_Con]@Pg, GRB.MINIMIZE)

# calculate the optimization
model.optimize()
model.X[:14]

Set parameter NonConvex to value 2
ipol: [0 1 2]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-10600KF CPU @ 4.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 34 rows, 46 columns and 152 nonzeros
Model fingerprint: 0x47ad00e0
Model has 22 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e-16, 1e+04]
  RHS range        [1e+00, 7e+01]
  QRHS range       [1e+00, 2e+03]
Presolve removed 4 rows and 4 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 12 rows and 14 columns
Presolve time: 0.00s
Presolved: 85 rows, 50 columns, 292 nonzeros
Presolved model has 11 quadratic constraint(s)
Presolved model has 18 bilinear constraint(s)
Variable types: 50 continuous, 0 integer (0 binary)

[1.05,
 1.0106317566373713,
 1.017788190556323,
 1.0164889138132425,
 0.0,
 0.018627900517429623,
 0.0671356757219666,
 0.0845593760423413,
 1.798612909896436e-10,
 79.99999998990722,
 77.02045375909742,
 16.144166734275736,
 -2.2741022318031097,
 -17.57565640134782]